In [1]:
import torch
import torch.nn as nn
import os

from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.chrome.options import Options
import numpy as np 
import pandas as pd
import datetime
import pickle
import time

/Users/alexlee/Library/Python/3.8/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
driver = Chrome()
driver.get("https://racing.hkjc.com/racing/information/English/Racing/RaceCard.aspx")
WebDriverWait(driver, 10, 1).until(lambda x: x.find_element(By.ID, 'hplnkColSelect').is_displayed())

driver.find_element(By.ID, 'hplnkColSelect').click()

driver.find_element(By.XPATH, '//*[@id="ColSelectBody"]/form/table/tbody/tr[2]/td[3]/input').click()
driver.find_element(By.XPATH, '//*[@id="ColSelectBody"]/form/table/tbody/tr[2]/td[4]/input').click()
driver.find_element(By.XPATH, '//*[@id="ColSelectBody"]/form/table/tbody/tr[4]/td[5]/input').click()
driver.find_element(By.XPATH, '//*[@id="ColSelectBody"]/form/table/tbody/tr[6]/td[2]/input').click()
driver.find_element(By.XPATH, '//*[@id="ColSelectBody"]/form/table/tbody/tr[10]/td[3]/input').click()

driver.find_element(By.XPATH, '//*[@id="ColSelectBody"]/form/table/thead/tr/td/table/tbody/tr/td[2]/a').click()
driver.find_element(By.ID, 'hplnkColSelect').click()


TimeoutException: Message: timeout: Timed out receiving message from renderer: 300.000
  (Session info: chrome=120.0.6099.129)
Stacktrace:
0   chromedriver                        0x0000000104198cc8 chromedriver + 4844744
1   chromedriver                        0x0000000104190243 chromedriver + 4809283
2   chromedriver                        0x0000000103d5e77d chromedriver + 411517
3   chromedriver                        0x0000000103d43c6d chromedriver + 302189
4   chromedriver                        0x0000000103d43ad8 chromedriver + 301784
5   chromedriver                        0x0000000103d421a5 chromedriver + 295333
6   chromedriver                        0x0000000103d42989 chromedriver + 297353
7   chromedriver                        0x0000000103d537ac chromedriver + 366508
8   chromedriver                        0x0000000103d6c498 chromedriver + 468120
9   chromedriver                        0x0000000103d727fb chromedriver + 493563
10  chromedriver                        0x0000000

In [ ]:
target_race_no = 1

In [ ]:
try:
    original_window = driver.current_window_handle
    for window_handle in driver.window_handles:
        if window_handle != original_window:
            driver.switch_to.window(window_handle)
            driver.close()
    driver.switch_to.window(original_window)
except:
    pass

driver.get("https://racing.hkjc.com/racing/information/English/Racing/RaceCard.aspx")
WebDriverWait(driver, 10, 1).until(lambda x: x.find_element(By.ID, 'hplnkColSelect').is_displayed())

race_buttons_indices = []

for cell_idx, cell in enumerate(driver.find_elements(By.XPATH, '//*[@id="innerContent"]/div[2]/div[3]/table/tbody/tr[1]/td')):
    if len(cell.find_elements(By.TAG_NAME, 'img')) > 0:
        race_buttons_indices.append(cell_idx + 1)

for race_button_idx in race_buttons_indices:
    time.sleep(0.5)
    driver.find_element(By.XPATH, f'//*[@id="innerContent"]/div[2]/div[3]/table/tbody/tr[1]/td[{race_button_idx}]').click()
    WebDriverWait(driver, 10, 1).until(lambda x: x.find_element(By.XPATH, '//*[@id="racecardlist"]/tbody/tr/td/table').is_displayed())

    race_details = driver.find_element(By.XPATH, '//*[@id="innerContent"]/div[2]/div[4]/div[2]').text.splitlines()
    race_no = race_details[0].split('-')[0].replace('Race','').strip()

    if int(race_no) != target_race_no:
        continue

    race_date = datetime.datetime.strptime(''.join(race_details[1].split(',')[1:3]).strip(), '%B %d %Y')
    race_surface = race_details[2].split(',')[0].strip().lower()
    race_location = race_details[1].split(',')[3].strip().lower()   

    path = os.path.join('data','6_prediction',f'{race_date.strftime("%Y_%m_%d")}_{race_no}.csv')
    df = pd.read_csv(path, index_col=0)

    if race_surface == 'All Weather Track':
        race_course = 'all weather track'
        race_distance = race_details[2].split(',')[1].replace('M','').strip().lower()
        try:
            race_going = race_details[2].split(',')[2].strip().lower()
        except:
            race_going = np.nan
    else:
        race_course = race_details[2].split(',')[1].replace('"','').replace('Course','').strip().lower()
        race_distance = race_details[2].split(',')[2].replace('M','').strip().lower()
        try:
            race_going = race_details[2].split(',')[3].strip().lower()
        except:
            race_going = np.nan

    num_horses = len(driver.find_elements(By.XPATH, '//*[@id="racecardlist"]/tbody/tr/td/table/tbody/tr'))
    num_horses_scraped = len(df.index)

    horse_handicap = [] 
    horse_weight = [] 
    horse_gear = []

    for horse_idx, horse_row in enumerate(driver.find_elements(By.XPATH, '//*[@id="racecardlist"]/tbody/tr/td/table/tbody/tr')):
        horse_row = driver.find_elements(By.XPATH, f'//*[@id="racecardlist"]/tbody/tr/td/table/tbody/tr[{horse_idx + 1}]/td')

        if num_horses_scraped < num_horses:
            if horse_row[5].text == '-':
                continue

        horse_handicap.append(horse_row[5].text)
        horse_weight.append(horse_row[13].text)
        horse_gear.append(int(horse_row[22].text.strip() != ''))
        
    df['race_going'] = race_going
    df['horse_handicap'] = horse_handicap
    df['horse_weight'] = horse_weight
    df['horse_gear'] = horse_gear

    # go to bet page and get odds
    # for _ in range(5):
    #     try:
    driver.get('https://bet.hkjc.com/racing/pages/odds_wp.aspx?lang=en')
    WebDriverWait(driver, 10, 1).until(lambda x: x.find_element(By.ID, 'winCell1').is_displayed())
            # time.sleep(1)
        # except:
        #     continue

    driver.find_element(By.ID, f'raceSel{race_no}').click()
    WebDriverWait(driver, 10, 1).until(lambda x: x.find_element(By.ID, 'winCell1').is_displayed())
    time.sleep(1)

    try:
        for option in driver.find_elements(By.XPATH, '//*[@id="container"]/div/div/div[2]/div[2]/div[1]/select/option'):
            if race_location in option.text.lower():
                option.click()
                WebDriverWait(driver, 10, 1).until(lambda x: x.find_element(By.ID, 'winCell1').is_displayed())
                break
    except:
        pass

    all_odds = []
    for horse_idx in range(num_horses):
        if num_horses_scraped < num_horses:
            try:
                float(driver.find_element(By.ID, f'winCell{horse_idx + 1}').text)
            except:
                continue
        all_odds.append(driver.find_element(By.ID, f'winCell{horse_idx + 1}').text)
    df['horse_odds'] = all_odds

    df['horse_odds'] = pd.to_numeric(df['horse_odds'], errors='coerce')
    df = df[df['horse_odds'].notna()]

    df.to_csv(path)

    driver.get("https://racing.hkjc.com/racing/information/English/Racing/RaceCard.aspx")
    WebDriverWait(driver, 10, 1).until(lambda x: x.find_element(By.ID, 'hplnkColSelect').is_displayed())

driver.close()

IndexError: list index out of range